# Introdução

Esse notebook traz as analises pedidas na disciplina Biologia Evolutiva - Bio507

Vamos aproveitar para ilustrar o uso de python e R dentro do mesmo ambiente de trabalho.

## Leitura de dados

Primeiro vamos ler os dados usando o pandas.
A função read_csv cria um data.frame com os dados.

Podemos usar a função describe para ter um resumo global.

In [1]:
import pandas as pd
import numpy as np
import dendropy  as dp

dados_brutos = pd.read_csv("./dados.csv")
num_traits = 4
traits = dados_brutos.columns[:num_traits]
dados_brutos

ImportError: No module named 'pandas'

In [ ]:
dados_brutos.head()

In [ ]:
dados_brutos.describe()

Podemos também usar a função groupby para aplicar funções em sub conjuntos dos dados, agrupando pela coluna ESPECIE

In [ ]:
dados_brutos.groupby('ESPECIE').describe()

Vamos calcular as médias, desvios padrão e coeficiantes de variação por traço por espécie.

In [ ]:
medias = dados_brutos.groupby('ESPECIE').mean()
medias

In [ ]:
std = dados_brutos.groupby('ESPECIE').std()
std

In [ ]:
cv = std/medias
cv

###Usando o R

Também é possivel usar diretamente o R dentro do notebook.

Vamos usar o pacote ggplot2 para fazer um gráfico sumário das nossas variáveis.

In [ ]:
%load_ext rmagic 

In [ ]:
import pandas.rpy.common as com
dados_brutos_R = com.convert_to_r_dataframe(dados_brutos)
%Rpush dados_brutos_R

In [ ]:
%%R
library(ggplot2); library(reshape2)
dados_brutos_R[1:4] = apply(dados_brutos_R[1:4], 2, as.numeric)
dados_brutos_R[,5] = as.character(dados_brutos_R[,5])
dados.melted = melt(dados_brutos_R, id.vars = 'ESPECIE')
box_plots = ggplot(dados.melted, 
                   aes(ESPECIE, 
                       value, 
                       group = interaction(ESPECIE, variable), 
                       color= variable)) + geom_boxplot()
print(box_plots)

##Calculando matrizes de covariância

Vamos calcular as matrizes de covariância e correlação por espécie, além das médias, e agrupá-las num dicionário.

In [ ]:
cov_matrices = dados_brutos.groupby('ESPECIE').apply(lambda x: x.cov())
cor_matrices = dados_brutos.groupby('ESPECIE').apply(lambda x: x.corr())
especies_labels = list(pd.unique(dados_brutos['ESPECIE']))

In [ ]:
cov_matrices

In [ ]:
cor_matrices

Podemos também acessar as matrizes pelo nome da espécie:

In [ ]:
cor_matrices.T['C']

Ou fazer um plot de todas elas

In [ ]:
%R library(plotrix)
for sp in especies_labels:
    mat = np.array(cor_matrices.T[sp])
    print sp
    %Rpush mat
    %R color2D.matplot(mat)

##Analise de variância

Calculando as matrizes dentro de grupos, entre grupos e total

In [ ]:
%%R 

library(Morphometrics)

n.total = dim(dados_brutos_R)[1]
fivesp_lm = lm(as.matrix(dados_brutos_R[,1:4])~dados_brutos_R[,5])
fivesp_lm_total = lm(as.matrix(dados_brutos_R[,1:4])~1)

SSCP.W = t(fivesp_lm$residuals)%*%fivesp_lm$residuals
SSCP.T = t(fivesp_lm_total$residuals)%*%fivesp_lm_total$residuals
SSCP.B = SSCP.T - SSCP.W
b_matrix = SSCP.B/n.total
w_matrix = CalculateMatrix(fivesp_lm)
t_matrix = CalculateMatrix(fivesp_lm_total)

In [ ]:
%Rpull b_matrix w_matrix t_matrix

##Morphometrics

Passando as matrizes pro R para uso do pacote Morphometrics

In [ ]:
cov_matrices_R = com.convert_to_r_dataframe(cov_matrices)
cor_matrices_R = com.convert_to_r_dataframe(cor_matrices)
%Rpush cov_matrices_R cor_matrices_R especies_labels

In [ ]:
%%R
library(Morphometrics)

especies <- unlist(especies_labels)
cov_matrices_R['ESPECIE'] <- cor_matrices_R['ESPECIE'] <- rep(especies, each = 4)
cov_matrices <- dlply(cov_matrices_R, .variables='ESPECIE', function(x) as.matrix(x[1:4]))
cor_matrices <- dlply(cor_matrices_R, .variables='ESPECIE', function(x) as.matrix(x[1:4]))
print(cov_matrices)

In [ ]:
%%R
reps <- laply(cov_matrices, MonteCarloRep, "random", 100)
print(reps)

In [ ]:
%%R 
RS <- RandomSkewers(cov_matrices, repeat.vector = reps)
print(RS)

##Filogenia

Vamos incluir uma filogenia e calcular estados ancestrais

In [ ]:
tree = dp.Tree.get_from_string("4(E, 3(2(C, B),1(A,D)))", "newick")
tree.print_plot(display_width = 50, show_internal_node_labels = True, leaf_spacing_factor = 4)

Esse código usa as matrizes e médias calculadas anteriormente, junto com os tamanhos amostrais, para calcular valores ponderados para todos os nós da filogenia. 

A conta realizada para médias e matrizes é uma simples média ponderada.

In [ ]:
get_node_name = lambda n: str(n.label or n.taxon or None)
nodes = [get_node_name(n) for n in tree.nodes()]

node_matrices = {}
node_sample_size = {}
for sp in especies_labels:
    new_matrix = np.array(cov_matrices.T[sp])
    node_matrices[sp] = new_matrix
    node_sample_size[sp] = dados_brutos[dados_brutos['ESPECIE'] == sp].shape[0]

# Tirando quem nao esta na filogenia e trocando os keys

node_means = {}

for sp in especies_labels:
    if tree.find_node_with_taxon_label(sp):
        new_key = get_node_name(tree.find_node_with_taxon_label(sp))
        node_means[new_key] = medias.T[sp]
        node_sample_size[new_key] = node_sample_size.pop(sp)
        node_matrices[new_key] = node_matrices.pop(sp)
    else:
        node_matrices.pop(sp)
        node_sample_size.pop(sp)

# Função que recebe uma lista de filhos e calcula a matriz, média e tamanho amostral pro ancestral

def ancestral_mean(child_labels):
    new_matrix = np.zeros((num_traits, num_traits))
    sample = 0
    new_mean = np.zeros(num_traits)
    for child in child_labels:
        node = get_node_name(child)
        new_matrix = new_matrix +\
            node_sample_size[node] * node_matrices[node]
        sample = sample + node_sample_size[node]
        new_mean = new_mean + node_sample_size[node] * node_means[node]
    new_matrix = new_matrix/sample
    new_mean = new_mean/sample
    return new_matrix, sample, new_mean

# Calculando as matrizes e tamanhos amostrais para todos os nós

for n in tree.postorder_node_iter():
    if get_node_name(n) not in node_matrices:
        node_matrices[get_node_name(n)], node_sample_size[get_node_name(n)], node_means[get_node_name(n)] = ancestral_mean(n.child_nodes())

Isso resulta num dicionário, chamado node_matrices, com todas as matrizes para todos os nós.

In [ ]:
node_matrices

Um dicionário de tamanhos amostrais

In [ ]:
node_sample_size

E um dicionário de médias.

In [ ]:
node_means['1']

Temos tb uma lista de todos os nós:

In [ ]:
nodes

É interessante notar como a matriz estimada para a raiz, ponderando as matrizes ao longo da filogenia, é idêntica à matriz ponderada dentro de grupos, calculada pelos modelos lineares anteriormente:

In [ ]:
w_matrix

In [ ]:
node_matrices['4']

### $\beta$ e $\Delta z$

Vamos agora estimar as mudanças evolutivas em cada ramo da filogenia, e, usando as matrizes ancestrais, calcular os gradientes de seleção estimados.

Os $\Delta z$ são apenas as diferenças nas médias de um nó com o seu acestral. Os $\beta$ são estimados com a equação de Lande:

$
\beta = G^{-1}\Delta z
$

In [ ]:
delta_z = {}
beta = {}
for n in tree.nodes()[1:]: #começamos do 1 para pular a raiz, que não tem ancestral
    parent = get_node_name(n.parent_node)
    branch = get_node_name(n) + '_' + parent
    delta_z[branch] = node_means[get_node_name(n)] - node_means[parent]
    beta[branch] = np.linalg.solve(node_matrices[parent], delta_z[branch])

In [ ]:
delta_z

In [ ]:
beta

Podemos agora calcular a correlação entre os $\beta$ e $\Delta z$

In [ ]:
def vector_corr(x, y): return (np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y)))
corr_beta_delta_z = {}
for branch in delta_z:
    corr_beta_delta_z[branch] = vector_corr(beta[branch], delta_z[branch])

In [ ]:
corr_beta_delta_z

Podemos também calcular a relação entre a resposta evolutiva e o primeiro componente principal da matriz de covariação, a linha de menor resistência evolutiva. 

Como a direção primeiro componente é arbitrária, tomamos o valor absoluto da correlação.

In [ ]:
corr_pc1 = {}
for branch in delta_z:
    parent = branch.split("_")[1]
    pc1 = np.linalg.eig(node_matrices[parent])[1][:,0]
    corr_pc1[branch] = abs(vector_corr(delta_z[branch], pc1))

In [ ]:
corr_pc1

Podemos utilizar o pandas para formatar esses resultados em tabelas

In [ ]:
df_betas = pd.DataFrame.from_dict(beta, orient='index')
df_betas.columns = traits
df_betas

In [ ]:
df_dz = pd.DataFrame.from_dict(delta_z, orient='index')
df_dz.columns = traits
df_dz

In [ ]:
traits_c = list(traits)
traits_c.append('otu')
df_matrices = pd.DataFrame(columns=traits_c)
for node in node_matrices:
    df = pd.DataFrame(node_matrices[node], columns=traits, index = traits)
    df['otu'] = node
    df_matrices = df_matrices.append(df)
df_matrices
